## Лаба 10. Построить рекомендательную систему видеоконтента с implicit feedback

<img width="350px" src="images/megafon_logo.jpg">

##### Лаба создана при поддержке компании «МегаФон». 

### Дедлайн

⏰ Четверг, 20 июня 2019 года, 23:59.

### Задача

В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами
покупок абонентами телепередач от компании E-Contenta. По доступным вам данным нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами.

### Обработка данных на вход

Для выполнения работы вам следует взять все файлы из папки на HDFS `/labs/lab10data/`. Давайте посмотрим, что у нас есть:

```
$ hadoop fs -ls /labs/lab10data
Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2017-05-09 13:51 /labs/lab10data/lab10_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2017-05-09 13:50 /labs/lab10data/lab10_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2017-05-09 13:50 /labs/lab10data/lab10_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2017-05-09 13:51 /labs/lab10data/lab10_views_programmes.csv
```

* В `lab10_train.csv` содержатся факты покупки (колонка `purchase`) пользователями (колонка `user_id`) телепередач (колонка `item_id`). Такой формат файла вам уже знаком.

* `lab10_items.csv` — дополнительные данные по items. В данном файле много лишней или ненужной информации, так что задача её фильтрации и отбора ложится на вас. Поля в файле, на которых хотелось бы остановиться:
  * `item_id` — primary key. Соответствует `item_id` в предыдущем файле.
  * `content_type` — тип телепередачи (`1` — платная, `0` — бесплатная). Вас интересуют платные передачи.
  * `title` — название передачи, текстовое поле.
  * `year` — год выпуска передачи, число.
  * `genres` — поле с жанрами передачи, разделёнными через запятую.
* `lab10_test.csv` — тестовый датасет без указанного целевого признака `purchase`, который вам и предстоит предсказать.
* Дополнительный файл `lab10_views_programmes.csv` по просмотрам передач с полями:
  * `ts_start` — время начала просмотра
  * `ts_end` — время окончания просмотра
  * `item_type`— тип просматриваемого контента:
    * `live` — просмотр "вживую", в момент показа контента в эфире
    * `pvr` — просмотр в записи, после показа контента в эфире


### Обработка данных на выход

Предсказание целевой переменной "купит/не купит" — хорошо знакомая вам задача бинарной классификации, с которой вы уже встречались в [Лабе 4](../../labs/lab04/lab04.md). Поскольку нам важны именно вероятности отнесения пары `(пользователь, товар)` к классу "купит" (`1`), то, на самом деле, вы можете подойти к проблеме с разных сторон:
1. Как к разработке рекомендательной системы: рекомендовать пользователю `user_id` топ-N лучших телепередач, которые были найдены по методике user-user / item-item коллаборативной фильтрации.
2. Как к задаче факторизации матриц: алгоритмы SVD, ALS, FM/FFM.
3. Как просто к задаче бинарной классификации. У вас есть два датасета, которые можно каким-то образом объединить, дополнительно обработать и сделать предсказания классификаторами (Apache Spark, pandas + sklearn на ваше усмотрение).
4. Как к задаче регрессии. Поскольку от вас требуется предсказать не факт покупки, а его *вероятность*, то можно перевести задачу в регрессионную и решать её соответствующим образом. 

### Подсказки

1. Кроссвалидация — ваш друг. Используйте `pyspark.ml.tuning.TrainValidationSplit` вместе с `ParamGridBuilder`, чтобы произвести grid search гиперпараметров вашей модели.
2. Простой подсчёт ROC AUC в Apache Spark доступен в `pyspark.ml.evaluation.BinaryClassificationEvaluator`.

### Проверка

Мы будем оценивать ваш алгоритм по метрике ROC AUC. Ещё раз напомним, что чекеру требуются *вероятности* в диапазоне `[0.0, 1.0]` отнесения пары `(пользователь, товар)` в тестовой выборке к классу "1" (купит).

Для успешного прохождения лабораторной работы **AUC должен составить не менее 0.6**.

**Важно!** Для точной проверки не забудьте отсортировать полученный файл по возрастанию идентификаторов пользователей (`user_id`), а затем — по возрастанию идентификаторов передач (`item_id`). Образец - `lab10_test.csv`

Результат следует сохранить в файл `lab10.csv` в своей домашней директории.

Проверка осуществляется [автоматическим скриптом](http://lk.newprolab.com/lab/laba10) из Личного кабинета.

## Решение

In [1]:
# Запуск pyspark
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
sc.setCheckpointDir('checkpoint/')

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Load and Review Data

In [4]:
!hadoop fs -ls /labs/lab10data

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2018-11-27 16:00 /labs/lab10data/lab10_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2018-11-27 16:00 /labs/lab10data/lab10_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2018-11-27 16:00 /labs/lab10data/lab10_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2018-11-27 16:00 /labs/lab10data/lab10_views_programmes.csv


#### lab10_train.csv

In [4]:
schema = StructType(fields=[StructField("user_id", IntegerType()),
                            StructField("item_id", IntegerType()),
                            StructField("purchase", IntegerType())])

train = spark.read.csv('/labs/lab10data/lab10_train.csv', schema=schema, header=True)
train.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [8]:
%time train.summary().show()

+-------+------------------+-----------------+--------------------+
|summary|           user_id|          item_id|            purchase|
+-------+------------------+-----------------+--------------------+
|  count|           5032624|          5032624|             5032624|
|   mean| 869680.9464782189|66869.30485865823|0.002166662957534...|
| stddev|60601.098215631355|35242.28205538276| 0.04649697795291635|
|    min|              1654|              326|                   0|
|    25%|            846231|            65667|                   0|
|    50%|            885247|            79853|                   0|
|    75%|            908588|            93606|                   0|
|    max|            941450|           104165|                   1|
+-------+------------------+-----------------+--------------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 15.9 s


In [13]:
# Количество уникальных юзеров
train.agg(countDistinct("user_id")).show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                   1941|
+-----------------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.49 s


In [14]:
# Количество уникальных шоу
train.agg(countDistinct("item_id")).show()

+-----------------------+
|count(DISTINCT item_id)|
+-----------------------+
|                   3704|
+-----------------------+

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.6 s


In [19]:
# Количество покупок
%time train.groupBy("purchase").count().show()

+--------+-------+
|purchase|  count|
+--------+-------+
|       1|  10904|
|       0|5021720|
+--------+-------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 10.9 s


In [18]:
train.rdd.getNumPartitions()

2

#### lab10_test.csv

In [5]:
schema = StructType(fields=[StructField("user_id", IntegerType()),
                            StructField("item_id", IntegerType()),
                            StructField("purchase", IntegerType())])

test = spark.read.csv('/labs/lab10data/lab10_test.csv', schema=schema, header=True)
test.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



In [11]:
%time test.summary().show()

+-------+-----------------+------------------+--------+
|summary|          user_id|           item_id|purchase|
+-------+-----------------+------------------+--------+
|  count|          2156840|           2156840|       0|
|   mean|869652.3733920922| 66896.00283609354|    null|
| stddev|60706.51616335023|35227.831307045984|    null|
|    min|             1654|               326|    null|
|    25%|           846164|             65668|    null|
|    50%|           885124|             79856|    null|
|    75%|           908588|             93606|    null|
|    max|           941450|            104165|    null|
+-------+-----------------+------------------+--------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 10.7 s


In [15]:
# Те же самые юзеры
test.agg(countDistinct("user_id")).show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                   1941|
+-----------------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.19 s


In [16]:
# Те же самые шоу
test.agg(countDistinct("item_id")).show()

+-----------------------+
|count(DISTINCT item_id)|
+-----------------------+
|                   3704|
+-----------------------+

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.83 s


In [17]:
test.rdd.getNumPartitions()

3

## ALS

In [6]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [7]:
# define evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="purchase", metricName="areaUnderROC")

#### Journal

maxIter | regParam | rank | alpha | nonnegative | rocauc_train | rocauc_test | max_test_conf | time | timestamp
:--- | :---: | :---: | :---: | :---: | :---: | :--- | :--- | :---
10 | 0.1 | 10 | 1.0 |False | 0.9671 | 0.7778 | 0.97? | 28s | 2019-14-06 12:29
10 | 1.0 | 10 | 1.0 | True | 0.9637  | 0.7986 | 0.27 | 28s | 2019-14-06 19:31 
10 | 1.0 | 10 | 40.0 | True | 0.9870 | 0.7885 | 2.31  | 28s | 2019-14-06 20:34
10 | 1.0 | 10 | 10.0 | True | 0.9795 | 0.8002  | 1.13 | 28s | 2019-14-06 21:22
10 | 10.0 | 10 | 10.0 | True | 0.8888 | 0.7381  | 0.005 | 28s | 2019-14-06 22:03
10 | 2.0 | 10 | 10.0 | True | 0.9788 | **0.8058**  | 0.82 | 28s | 2019-14-06 22:42
10 | 3.0 | 10 | 10.0 | True | 0.9764 | 0.8046  | 0.63 | 16s | 2019-14-06 22:52
10 | 2.0 | 20 | 10.0 | True | 0.9888 | 0.8041  | 0.83 | 13s | 2019-14-06 23:05
10 | 3.0 | 20 | 10.0 | True | 0.9878 | 0.8053  | 0.65 | 8s | 2019-14-06 23:13
10 | 3.0 | 30 | 10.0 | True | 0.9916 | 0.8026  | 0.66 | 7s | 2019-14-06 23:20
10 | 4.0 | 30 | 10.0 | True | 0.9905 | 0.8029  | 0.51 | 7s | 2019-14-06 23:26
10 | 4.0 | 40 | 10.0 | True | 0.9927 | 0.8035  | 0.48 | 7s | 2019-14-06 23:34
10 | 4.0 | 50 | 20.0 | True | 0.9974 | 0.8011  | 0.81 | 8s | 2019-14-06 23:41
10 | 5.0 | 50 | 20.0 | True | 0.9972 | 0.8012  | 0.72 | 8s | 2019-14-06 23:46
10 | 6.0 | 50 | 20.0 | True | 0.9968 | 0.8009  | 0.65 | 8s | 2019-14-06 23:52
20 | 6.0 | 50 | 20.0 | True | 0.9969 | 0.8008  | 0.66 | 10s | 2019-14-06 23:58
20 | 6.0 | 60 | 20.0 | True | 0.9975 | 0.8013  | 0.66 | 13s | 2019-15-06 00:04
20 | 6.0 | 70 | 20.0 | True | 0.9980 | 0.8012  | 0.68 | 13s | 2019-15-06 00:13
20 | 2.0 | 10 | 10.0 | True | 0.9788 | 0.8084  | 0.87 | 35s | 2019-15-06 23:26
20 | 2.0 | 10 | 10.0 | False | 0.9840 | 0.7952  | 0.83/clamp | 14s | 2019-15-06 23:35
20 | 2.0 | 10 | 10.0 | False | 0.9840 | 0.8189  | 0.83 | 14s | 2019-15-06 23:38
20 | 2.0 | 8 | 10.0 | False | 0.9801 | 0.8208  | 0.81 | 14s | 2019-15-06 23:48
20 | 2.0 | 7 | 10.0 | False | 0.9776 | 0.8210  | 0.80 | 42s | 2019-16-06 19:25
20 | 2.0 | **6** | 10.0 | False | 0.9739 | **0.8286**  | 0.83 | 23s | 2019-16-06 19:36
20 | 2.0 | 5 | 10.0 | False | 0.9690 | 0.8198  | 0.80 | 18s | 2019-16-06 19:46
20 | 2.0 | 6 | **5.0** | False | 0.9687 | **0.8330**  | 0.49 | 18s | 2019-16-06 19:58
20 | 2.0 | 6 | 4.0 | False | 0.9672 | 0.8315  | 0.37 | 1m6s | 2019-16-06 21:57
20 | 2.0 | 6 | 6.0 | False | 0.9699 | 0.8315  | 0.58 | 23s | 2019-16-06 22:08
20 | 2.5 | 6 | 5.0 | False | 0.9680 | 0.8325  | 0.37 | 16s | 2019-16-06 22:56
20 | 1.5 | 6 | 5.0 | False | 0.9692 | 0.8289  | 0.63 | 55s | 2019-16-06 23:07
**20** | **2.2** | 6 | 5.0 | False | 0.9685 | **0.8334**  | 0.44 | 27s | 2019-16-06 23:17
15 | 2.2 | 6 | 5.0 | False | 0.9685 | 0.8323  | 0.44 | 37s | 2019-16-06 23:33
10 | 2.2 | 6 | 5.0 | False | 0.9689 | 0.8319  | 0.43 | 18s | 2019-16-06 23:41
30 | 2.2 | 6 | 5.0 | False | 0.9681 | 0.8334  | 0.44 | 31s | 2019-16-06 23:49

In [219]:
# Fit ALS on the training data
als = ALS(maxIter=20, regParam=2.2, rank=6, coldStartStrategy="nan", \
          userCol='user_id', itemCol='item_id', ratingCol='purchase', \
          nonnegative=False, implicitPrefs=True, alpha=5.0, seed=87)
%time als_model = als.fit(train)

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 31.3 s


In [220]:
#Let see how the model perform on train set
predict_train = als_model.transform(train)
%time predict_train.show(5)

+-------+-------+--------+-------------+
|user_id|item_id|purchase|   prediction|
+-------+-------+--------+-------------+
| 746713|   8389|       0|          0.0|
| 883098|   8389|       0|-0.0036029667|
| 903491|   8389|       0| 0.0056212842|
| 903826|   8389|       0|  0.037457652|
| 916566|   8389|       0| 5.723099E-27|
+-------+-------+--------+-------------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 17.2 s


In [221]:
predict_train.rdd.getNumPartitions()

200

In [222]:
predict_train = predict_train.coalesce(4).cache()

In [223]:
predict_train.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- prediction: float (nullable = false)



In [224]:
# We need DoubleType() in prediction column for the evaluator
predict_train = predict_train.withColumn("prediction", predict_train.prediction.cast(DoubleType()))
%time predict_train.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- prediction: double (nullable = false)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 329 µs


In [226]:
%time predict_train.checkpoint()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 32.4 s


DataFrame[user_id: int, item_id: int, purchase: int, prediction: double]

In [227]:
%time predict_train.summary().show()

+-------+-----------------+------------------+--------------------+--------------------+
|summary|          user_id|           item_id|            purchase|          prediction|
+-------+-----------------+------------------+--------------------+--------------------+
|  count|          5032624|           5032624|             5032624|             5032624|
|   mean|869680.9464782189| 66869.30485865823|0.002166662957534...| 0.00511500408708944|
| stddev|60601.09821563049|35242.282055382544|0.046496977952915644|0.019791464685331724|
|    min|             1654|               326|                   0|-0.21284088492393494|
|    25%|           846231|             60351|                   0|-2.35493163927458...|
|    50%|           885247|             79853|                   0|                 0.0|
|    75%|           908726|             93602|                   0|0.003300537588074...|
|    max|           941450|            104165|                   1| 0.45940831303596497|
+-------+------------

In [228]:
# check roc_auc on the train set
%time rocauc_train = evaluator.evaluate(predict_train)
print(f'ROC AUC for train data: {rocauc_train}')

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 10.8 s
ROC AUC for train data: 0.9681270722320425


In [229]:
# predict test data
predict_test = als_model.transform(test)
%time predict_test.show(5)

+-------+-------+--------+-------------+
|user_id|item_id|purchase|   prediction|
+-------+-------+--------+-------------+
| 740405|   8389|    null|-5.8040954E-4|
| 838617|   8389|    null| 0.0054843277|
| 916910|   8389|    null|-0.0045700995|
| 814235|   8389|    null| 6.6968426E-4|
| 849754|   8389|    null| -6.556209E-4|
+-------+-------+--------+-------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 s


In [230]:
predict_test.rdd.getNumPartitions()

200

In [231]:
predict_test = predict_test.coalesce(4).cache()

In [232]:
%time predict_test.summary().show()

+-------+-----------------+-----------------+--------+--------------------+
|summary|          user_id|          item_id|purchase|          prediction|
+-------+-----------------+-----------------+--------+--------------------+
|  count|          2156840|          2156840|       0|             2156840|
|   mean|869652.3733920922|66896.00283609354|    null|0.005011935678396624|
| stddev|60706.51616333814|35227.83130704636|    null| 0.01915306531477629|
|    min|             1654|              326|    null|         -0.20226234|
|    25%|           846231|            65667|    null|       -2.3715185E-4|
|    50%|           885247|            79856|    null|                 0.0|
|    75%|           908726|            93606|    null|        0.0032955278|
|    max|           941450|           104165|    null|          0.44184247|
+-------+-----------------+-----------------+--------+--------------------+

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 16.3 s


## Write Output as csv to hdfs & copy to local

In [233]:
# make output dataframe
output = predict_test.select('user_id', 'item_id', col('prediction').alias('purchase')) \
                     .orderBy(['user_id', 'item_id'])
output.show(5)

+-------+-------+------------+
|user_id|item_id|    purchase|
+-------+-------+------------+
|   1654|    336|         0.0|
|   1654|    678|         0.0|
|   1654|    691|         0.0|
|   1654|    696| 7.360736E-4|
|   1654|    763|0.0017997124|
+-------+-------+------------+
only showing top 5 rows



In [234]:
output.rdd.getNumPartitions()

200

In [235]:
# write csv file with predictions
%time output.coalesce(1).write.csv('/user/sergey.zaytsev/lab10', header=True, sep=',', mode='overwrite')

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 5.77 s


In [236]:
!hadoop fs -ls /user/sergey.zaytsev/lab10

Found 2 items
-rw-r--r--   3 sergey.zaytsev sergey.zaytsev          0 2019-06-16 23:48 /user/sergey.zaytsev/lab10/_SUCCESS
-rw-r--r--   3 sergey.zaytsev sergey.zaytsev   50180152 2019-06-16 23:48 /user/sergey.zaytsev/lab10/part-00000-e431069f-4b6b-4ebd-b133-9e64c94d8d80-c000.csv


In [237]:
# Copy output file to local directory
!hadoop fs -copyToLocal /user/sergey.zaytsev/lab10/* ~/

## Shut down Spark

In [238]:
sc.stop()